In [1]:
import tensorflow as tf
import numpy as np
from Dataset import Dataset
from convnet import *

In [2]:
#Testing TF GPU Support

# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
test_sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(test_sess.run(c))
test_sess.close()

[[ 22.  28.]
 [ 49.  64.]]


In [24]:
sess = None
X = np.load("./Data/np/7/images_0.npy")
Y = np.load("./Data/np/7/labels_0.npy")

# Fixed width/height/depth at 96x96x96
X = np.pad(X, [[0,0], [3,3], [11, 12], [11, 12]], 'constant', constant_values=0)

# Network Parameters
data_shape = [X.shape[1], X.shape[2], X.shape[3]]
n_input = X.shape[1] * X.shape[2] * X.shape[3]   # Input size
n_classes = 2     # Classes (No Emphysema, Emphysema)
dropout = 0.75    # Dropout, probability to keep units

# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 2
display_step = 10

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

print(X.shape)
    
ds = Dataset(X, Y)

(92, 96, 96, 96)


In [31]:
# Number of neurons = Layer Output Size 
# Number of biases = 1 Per neuron
# Number of weights = FilterWidth*Fitlerheight*FiltherDepth*InputColors

# Store layers weight & bias
weights = {
    # 4x4x4 conv, 1 input, 96 outputs
    'wc1': tf.Variable(tf.random_normal([4, 4, 4, 1, 96])),
    # 4x4x4 conv, 48 inputs, 48 outputs
    'wc2': tf.Variable(tf.random_normal([4, 4, 4, 48,96])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*48, 1024])),
    # 1024 inputs, 2 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}


biases = {
    'bc1': tf.Variable(tf.random_normal([96])),
    'bc2': tf.Variable(tf.random_normal([96])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, data_shape, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

if sess:
    sess.close()
    
sess = tf.Session()

ValueError: Dimensions 96 and 48 are not compatible

In [26]:
# Launch the graph
with sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = ds.next_batch(batch_size)
        print(batch_y.shape)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print( "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    #print "Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
    #                                  y: mnist.test.labels[:256],
    #                                  keep_prob: 1.})



(2, 2)


InternalError: cuDNN launch failure : input shape([2,1,97,97,97]) filter shape([4,4,4,1,96])
	 [[Node: Conv3D_15 = Conv3D[T=DT_FLOAT, padding="SAME", strides=[1, 1, 1, 1, 1], _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape_15, Variable_64/read)]]
Caused by op 'Conv3D_15', defined at:
  File "/home/mostafa/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mostafa/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mostafa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/mostafa/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/mostafa/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/home/mostafa/anaconda3/lib/python3.5/site-packages/zmq/event